1. Python3
2. Pytorch

In [49]:
import numpy as np
import pickle
import copy
%matplotlib inline
import matplotlib.pyplot as plt
import torch
import torch.utils.data

class Loader:
    
    def unpickle(self,file):
        with open(file, 'rb') as fo:
            dict = pickle.load(fo, encoding='bytes')
        return dict
    
    def load_train_data(self):
        '''
        loads training data: 50,000 examples with 3072 features
        '''
        X_train = None
        Y_train = None
        for i in range(1, 6):
            pickleFile = self.unpickle('cifar-10-batches-py/data_batch_{}'.format(i))
            dataX = pickleFile[b'data']
            dataY = pickleFile[b'labels']
            
            if type(X_train) is np.ndarray:
                
                X_train = np.concatenate((X_train, dataX))
                Y_train = np.concatenate((Y_train, dataY))
            else:
                
                X_train = dataX
                Y_train = dataY
        
        X_train = X_train.reshape([50000, 3, 32, 32])
        X_train = X_train.astype(np.float32)
        
        X_train[:,0,:,:] = (X_train[:,0,:,:] - np.mean(X_train[:,0,:,:])) / np.std(X_train[:,0,:,:])
        X_train[:,1,:,:] = (X_train[:,1,:,:] - np.mean(X_train[:,1,:,:])) / np.std(X_train[:,1,:,:])
        X_train[:,2,:,:] = (X_train[:,2,:,:] - np.mean(X_train[:,2,:,:])) / np.std(X_train[:,2,:,:])

        
        Y_train = Y_train.reshape(Y_train.shape[0], 1)

        return X_train, Y_train.reshape((50000,))

    def load_test_data(self):
        '''
        loads testing data: 10,000 examples with 3072 features
        '''
        X_test = None
        Y_test = None
        pickleFile = self.unpickle('cifar-10-batches-py/test_batch')
        dataX = pickleFile[b'data']
        dataY = pickleFile[b'labels']
        if type(X_test) is np.ndarray:
            X_test = np.concatenate((X_test, dataX))
            Y_test = np.concatenate((Y_test, dataY))
        else:
            X_test = np.array(dataX)
            Y_test = np.array(dataY)

        X_test = X_test.reshape([10000, 3, 32, 32])
        X_test = X_test.astype(np.float32)
        
        X_test[:,0,:,:] = (X_test[:,0,:,:] - np.mean(X_test[:,0,:,:])) / np.std(X_test[:,0,:,:])
        X_test[:,1,:,:] = (X_test[:,1,:,:] - np.mean(X_test[:,1,:,:])) / np.std(X_test[:,1,:,:])
        X_test[:,2,:,:] = (X_test[:,2,:,:] - np.mean(X_test[:,2,:,:])) / np.std(X_test[:,2,:,:])

        
        Y_test = Y_test.reshape(Y_test.shape[0], 1)

        return X_test, Y_test.reshape((10000,))

In [50]:
X_train,Y_train = Loader().load_train_data()
X_test, Y_test= Loader().load_test_data()

print("X_Train: {} -> {} examples, {} features".format(X_train.shape, X_train.shape[1], X_train.shape[0]))
print("X_Test: {} -> {} examples, {} features".format(X_test.shape, X_test.shape[1], X_test.shape[0]))

X_Train: (50000, 3, 32, 32) -> 3 examples, 50000 features
X_Test: (10000, 3, 32, 32) -> 3 examples, 10000 features


In [51]:
x_train = X_train.copy()
y_train = Y_train.copy()

x_train = torch.from_numpy(x_train)  
y_train = torch.from_numpy(y_train) 

x_test = X_test.copy()
y_test = Y_test.copy()

x_test = torch.from_numpy(x_test)
y_test = torch.from_numpy(y_test)

train = []
valid = []
for i in range(50000):
    U = np.random.uniform()
    if U <= 0.2:
        valid.append((x_train[i],y_train[i]))
    else:
        train.append((x_train[i],y_train[i]))
    
test = []
for i in range(10000):
    test.append((x_test[i], y_test[i]))
    
trainloader = torch.utils.data.DataLoader(train, batch_size=20,  
                                          shuffle=True, num_workers=2)  

validloader = torch.utils.data.DataLoader(valid, batch_size=20,  
                                          shuffle=True, num_workers=2) 

testloader = torch.utils.data.DataLoader(test, batch_size=20,  
                                          shuffle=True, num_workers=2)

In [59]:
from torch.autograd import Variable  
import torch.nn as nn  
import torch.nn.functional as F  
  
class Net(nn.Module):               
    def __init__(self):      
        super(Net, self).__init__()           
        self.conv1 = nn.Conv2d(3, 32, 5)   
        self.pool = nn.MaxPool2d(2) 
        self.conv2 = nn.Conv2d(32, 64, 5)  
        self.avgpool = nn.AvgPool2d(2)
        
        self.conv3 = nn.Conv2d(64, 32, 3)
        
        self.fc1 = nn.Linear(32 * 3 * 3, 200) 
        self.fc2 = nn.Linear(200, 100) 
        self.fc3 = nn.Linear(100, 10) 
        
        self.bn1 = nn.BatchNorm2d(200)
        self.bn2 = nn.BatchNorm2d(100)

 # 12， 32， 200， 100
    def forward(self, x):                
                                         
        x = self.pool(F.relu(self.conv1(x)))
        x = self.avgpool(F.relu(self.conv2(x)))
        x = self.conv3(x)
        x = F.relu(x)

        x = x.view(-1, 32 * 3 * 3)   
                                    
        x = F.relu(self.bn1(self.fc1(x)))
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.fc3(x)
        x = F.log_softmax(x, dim = 1)
        return x  

net = Net()  

In [60]:
import torch.optim as optim        

optimizer = optim.SGD(net.parameters(), lr=0.003, momentum=0.9)   
loss_plot = []
vali_loss = []
train_acc = []
lr=0.003

In [ ]:
for epoch in range(20):  
    losstrack = 0.0
    
    for i, data in enumerate(trainloader, 0): 
        inputs, labels = data   
        inputs, labels = Variable(inputs), Variable(labels)   
                                                        
        optimizer.zero_grad()                  
        outputs = net(inputs)                
        loss = F.nll_loss(outputs, labels)
        loss.backward()                     
        optimizer.step()                   
   
        losstrack += loss.data[0] 
        if i % 1000 == 999:    
            print("epoch ", epoch + 1, "iteration ", i, "loss ", losstrack / 1000)
            loss_plot.append(losstrack/1000)
            losstrack = 0.0 
            
            valiloss = 0.0  
            c = 0
            for j, data in enumerate(validloader, 0):
                inputs, labels = data
                inputs, labels = Variable(inputs), Variable(labels)
                outputs = net(inputs)                
                loss = F.nll_loss(outputs, labels)
                valiloss += loss.data[0]
                c += 1
            vali_loss.append(valiloss/c)
            valiloss = 0.0
              
    if epoch % 3 == 2:  
        lr *= 0.9
        optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)
        
        
    correct = 0   
    total = 0     
    for data in trainloader: 
        x, y = data  
        outputs = net(Variable(x)) 
        _, predicted = torch.max(outputs.data, 1)  
        total += y.size(0)         
        correct += (predicted == y).sum() 
        
    train_acc.append(100 * correct / total)
        